# Игра угадай число

*также смотри раздел [Технические данные о среде выполнения](#технические-данные-о-среде-выполнения) этого документа.

## Предпосылки

Для завершения модуля "PYTHON-8. Инструменты для Data Science" ставится задача требующая навыков владения инструментами (изложенными в модуле). Для этого предоставляется код, созданный к этому моменту в рамках модуля.

Давайте проверим его работоспособность.

### Импортируем функции модуля

In [1]:
from game_v2 import score_game
from game_v2 import random_predict

Здесь мы делаем доступными функции модуля game_v2:
- *score_game* - функция, выявляющая через [среднее арифмитическое](https://ru.wikipedia.org/wiki/Среднее_арифметическое) за сколько попыток переданная ей функция поиска находит результат;
- *random_predict* - функция поиска заданного числа [методом научного тыка](https://ru.wikipedia.org/wiki/Метод_проб_и_ошибок) без запоминания истории, возвращающая количество потребовавшихся попыток.

### Проверим работоспособность *random_predict* несколько раз

In [7]:
score_game(random_predict)
score_game(random_predict)
score_game(random_predict)
score_game(random_predict)
score_game(random_predict)

Ваш алгоритм угадывает число в среднем за:98 попыток
Ваш алгоритм угадывает число в среднем за:96 попыток
Ваш алгоритм угадывает число в среднем за:101 попыток
Ваш алгоритм угадывает число в среднем за:96 попыток
Ваш алгоритм угадывает число в среднем за:99 попыток


99

Как видим, количество требующихся для нахождения заданного числа попыток варьируется около 100.

## Постановка задачи

Взяв предоставленный код за отправную точку добиться следующего:
- новая программа должна достигать результата менее чем за 20 попыток;
- код программы должен быть предоставлен через GitHub;
- также, созданный для решения данной задачи код необходимо оформить соответствующим для GitHub образом;
- поскольку код создаётся на языке Python он должен соответствовать PEP8;
- как и любой другой код, необходимо, чтобы он был воспроизводим;
- **согласно пункту 2 условий задания, должна быть реализована функция которая:**
  - для каждой попытки использует случайное число из известного нам диапазона
  - начальный диапазон - от 1 до 100;
  - очередной диапазон определяется из предыдущего путём отсечения по информации - меньше или больше искомого числа выбранное на предыдущей попытке число.

*P.S.: поскольку последний пункт (выделенный жирным) не учитвался при составлении кода, то для его реализации добавлена дополнительная функция (см. раздел [Создание функции поиска методом тыка](#создание-функции-поиска-методом-тыка))*

МЕТРИКА КАЧЕСТВА

Результаты оцениваются по среднему количеству попыток при 1000 повторений.

## Выбор алгоритма

Возьмём за основу [алоритм бинарного поиска](https://ru.wikipedia.org/wiki/Двоичный_поиск). Следуя его правилам поиск числа от 1 до 100 не должен превысить 7 шагов.

Так, например, если загадано число 1, то следуя правилу не оставлять ниже выбранного для попытки числа меньше вариантов чем выше, получим следующие шаги:
1. 100/2 = 50 => 51
2. 50/2 = 25 => 26
3. 25/2 = 12.5 => 13
4. 12/2 = 6 => 7
5. 6/2 = 3 => 4
6. 3/2 = 1.5 => 2
7. 1/2 = 0.5 => 1

## Организация кода

Для обратной совместимости оставим предоставленный код в оригинальном виде. А новый код оформим в файле [*result_game.py*](result_game.py).

Предоставим информацию по задаче отредактировав файл [*README.md*](README.md).

Воспользуемся имеющимся Jupyter Notebook [*game.ipynb*](game.ipynb), чтобы более подробно зафиксировать процесс выполнения задачи.

## Создание функции поиска

Создадим функцию *binary_search*:

In [11]:
from math import log2

def binary_search(hidden_number=1):
    
    interval = (1, 101) # mathematially half-open left-closed interval - [1, 101)
    max_attempts_to_panic = int(1 + log2(len(range(*interval))))
    
    for attempt_index in range(max_attempts_to_panic):
        
        predict_number = int(sum(interval) / 2)
        
        if predict_number == hidden_number:
            return attempt_index+1

        if predict_number < hidden_number:
            interval = (predict_number+1, interval[1])
        elif hidden_number < predict_number:
            interval = (interval[0], predict_number)

    raise RuntimeError('Algorithm working not like expected')

И проверим её работоспособность уже имеющейся функцией *score_game*:

In [35]:
from game_v2 import score_game

score_game(binary_search)
score_game(binary_search)
score_game(binary_search)
score_game(binary_search)
score_game(binary_search)

Ваш алгоритм угадывает число в среднем за:5 попыток
Ваш алгоритм угадывает число в среднем за:5 попыток
Ваш алгоритм угадывает число в среднем за:5 попыток
Ваш алгоритм угадывает число в среднем за:5 попыток
Ваш алгоритм угадывает число в среднем за:5 попыток


5

Таким образом легко убедиться в значительном преимуществе функции *binary_search* перед изначальной *random_predict*.

Также, можно заметить, что функция *binary_search* использует самоограничение по количеству попыток. Это сделано для того для того, чтобы случайно внесённая (или не выявленная при создании функции) ошибка не приводила к бесконечному циклу.

Убедимся в полезности этого самоограничения.

In [56]:
binary_search(0)

RuntimeError: Algorithm working not like expected

Однако, если задать выходящее за диапазон число 101:

In [58]:
binary_search(101)

7

То мы увидим, что фунция отработала, но не ожиданным образом. А именно нашла число не входящее в диапазон.

Что очередной раз свидетельствует о необходимости проверки входных параметров. Доработаем функцию *binary_search*.

In [4]:
from math import log2

def binary_search(hidden_number=1):
    if not(1 <= hidden_number <= 100):
        raise ValueError('Expected value from 1 to 100')
    
    interval = (1, 101) # mathematially half-open left-closed interval - [1, 101)
    max_attempts_to_panic = int(1 + log2(len(range(*interval))))
    
    for attempt_index in range(max_attempts_to_panic):
        
        predict_number = int(sum(interval) / 2)
        
        if predict_number == hidden_number:
            return attempt_index+1

        if predict_number < hidden_number:
            interval = (predict_number+1, interval[1])
        elif hidden_number < predict_number:
            interval = (interval[0], predict_number)

    raise RuntimeError('Algorithm working not like expected')

Проверим снова:

In [60]:
binary_search(101)

ValueError: Expected value from 1 to 100

Теперь можно сказать, что функция работает как надо. И хотя необходимость проверки входных параметров на соответсвие типу никто не отменял, будем считать, что поставленная задача выполнена. Поскольку в реальных задачах проверка входных параметров истекает из логики построения кода и излишние проверки только замедлят выполнение кода поставленной задачи.

## Проверка функции поиска

Используя существующую функцию *score_game* мы убедились в качестве созданной функции *binary_search*. Задание, тем не менее гласит: "новая программа должна достигать результата менее чем за 20 попыток".

Создадим аналогичную *score_game* функцию для такой проверки и назовём её *reveal_max_attempts*.

In [5]:
import numpy as np

def reveal_max_attempts(search_function,
                       number_of_repetitions: int = 1000) -> int:
    
    hidden_numbers = np.random.randint(1, 101, size=number_of_repetitions)
    
    attempts_count = []
    for hidden_number in hidden_numbers:
        attempts_count.append(search_function(hidden_number))
        
    max_attempt_count = max(*attempts_count)
    print(f'Search algorithm needs maximum {max_attempt_count} attempts count')
    
    return max_attempt_count

И на практике убедимся в достаточности функции *binary_search*:

In [5]:
reveal_max_attempts(binary_search)
reveal_max_attempts(binary_search)
reveal_max_attempts(binary_search)
reveal_max_attempts(binary_search)
reveal_max_attempts(binary_search)

Search algorithm needs maximum 7 attempts count
Search algorithm needs maximum 7 attempts count
Search algorithm needs maximum 7 attempts count
Search algorithm needs maximum 7 attempts count
Search algorithm needs maximum 7 attempts count


7

## Оценка функции поиска

Ранее мы воспользовались предоставленной функцией *score_game* для оценки функции *binary_search*. Но, чтобы эта функциональность была доступна в создаваемом модуле, а также код соответствовал PEP8, создадим функцию *score_attempts*.

In [3]:
import numpy as np

def score_attempts(search_function,
                   number_of_repetitions: int = 1000) -> int:
    
    hidden_numbers = np.random.randint(1, 101, size=number_of_repetitions)
    
    attempts_count = []
    for hidden_number in hidden_numbers:
        attempts_count.append(search_function(hidden_number))
    
    score = int(np.mean(attempts_count))
    print(f'Search algorithm needs {score} attempts')
    
    return score

И воспользуемся ей:

In [7]:
score_attempts(binary_search)
score_attempts(binary_search)
score_attempts(binary_search)
score_attempts(binary_search)
score_attempts(binary_search)

Search algorithm needs 5 attempts
Search algorithm needs 5 attempts
Search algorithm needs 5 attempts
Search algorithm needs 5 attempts
Search algorithm needs 5 attempts


5

Как видно, количество шагов уменьшилось в 20 раз.

## Создание функции поиска методом тыка

Создадим функцию *random_binary_search*, которая совмещает в себе случайный выбор очередного числа и сужение диапазона поиска.

In [1]:
import numpy as np

def random_binary_search(hidden_number: int = 1) -> int:
    if not(1 <= hidden_number <= 100):
        raise ValueError('Expected value from 1 to 100')
    
    if not(1 <= hidden_number <= 100):
        raise ValueError('Expected value from 1 to 100')
    
    interval = (1, 101) # mathematially half-open left-closed interval - [1, 101)
    max_attempts_to_panic = 1 + len(range(*interval))
    
    for attempt_index in range(max_attempts_to_panic):
        
        predict_number = np.random.randint(*interval)

        if predict_number == hidden_number:
            return attempt_index+1

        if predict_number < hidden_number:
            interval = (predict_number+1, interval[1])
        elif hidden_number < predict_number:
            interval = (interval[0], predict_number)

    raise RuntimeError('Algorithm working not like expected')

## Оценка функции поиска методом тыка

### Согласно метрике качества

In [16]:
score_attempts(random_binary_search)
score_attempts(random_binary_search)
score_attempts(random_binary_search)
score_attempts(random_binary_search)
score_attempts(random_binary_search)

Search algorithm needs 7 attempts
Search algorithm needs 7 attempts
Search algorithm needs 7 attempts
Search algorithm needs 7 attempts
Search algorithm needs 7 attempts


7

### На максимальное количество попыток

In [28]:
reveal_max_attempts(random_binary_search)
reveal_max_attempts(random_binary_search)
reveal_max_attempts(random_binary_search)
reveal_max_attempts(random_binary_search)
reveal_max_attempts(random_binary_search)

Search algorithm needs maximum 16 attempts count
Search algorithm needs maximum 18 attempts count
Search algorithm needs maximum 16 attempts count
Search algorithm needs maximum 17 attempts count
Search algorithm needs maximum 17 attempts count


17

### Факты

- функция *random_binary_search* (основанная на методе тыка), как и функция *binary_search* (основанная на половинном делении), является сходящейся и выполняется за ограниченное количество шагов. В отличии от функции *random_predict*
- согласно метрике качества (функция *score_attempts*):
  - функция *binary_search* выполняется за 5 шагов
  - функция *random_binary_search* выполняется за 7 шагов
  - функция *random_predict* выполняется за 100 шагов
- согласно *reveal_max_attempts* (максимальное количество шагов в большинстве случаев):
  - функция *binary_search* - 7 шагов
  - функция *random_binary_search* - 18 шагов
  - функция *random_predict* - 850 шагов
- для реализации функции *random_binary_search* не требуется знание и воплощение специальных теорий. В отличии от функции *binary_search*

In [33]:
from game_v2 import random_predict

reveal_max_attempts(random_predict)
reveal_max_attempts(random_predict)
reveal_max_attempts(random_predict)
reveal_max_attempts(random_predict)
reveal_max_attempts(random_predict)

Search algorithm needs maximum 759 attempts count
Search algorithm needs maximum 832 attempts count
Search algorithm needs maximum 620 attempts count
Search algorithm needs maximum 670 attempts count
Search algorithm needs maximum 575 attempts count


575

### Выводы

- для достижения цели достаточна функция *random_binary_search*
- функция *binary_search* требует:
  - специальных знаний
  - дополнительного времени на их воплощение (написание и отладка кода)
- функция *binary_search* (по сравнению с функцией *random_binary_search*) не предлагает значительный прирост в большинстве случаев
- функция *random_binary_search* в большинстве случаев укладывается в 20 шагов

Я бы сравнил функции *binary_search* и *random_binary_search* как результат работы: 1) программиста по заданию аналитика, и; 2) дата сайентиста.

Т.е.:
- в качестве быстрого решения следует создать такую функцию, как *random_binary_search*
- а при необходимости и возможности можно создать специальную версию, такую как функция *binary_search*

## GitHub

Для того, чтобы опубликованый на GitHub код был доступен для понимания без необходимости его изучения, оформим поясняющий файл [*README.md*](README.md).

### Технические данные о среде выполнения

Для воспроизводимости, приведём информацию о системе, на которой разрабатывался код.

In [4]:
#cat /etc/lsb-release 
import subprocess
subprocess.run(["cat", "/etc/lsb-release"])

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=20.04
DISTRIB_CODENAME=focal
DISTRIB_DESCRIPTION="Ubuntu 20.04.4 LTS"


CompletedProcess(args=['cat', '/etc/lsb-release'], returncode=0)

In [5]:
#uname -r
import subprocess
subprocess.run(["uname", "-r"])

5.13.0-41-generic


CompletedProcess(args=['uname', '-r'], returncode=0)

In [7]:
import subprocess
#pip freeze
subprocess.run(['pip', 'freeze'])
#pip freeze > requirements.txt
with open('requirements.txt', 'w') as outfile:
    subprocess.run(['pip', 'freeze'], stdout=outfile, stderr=subprocess.STDOUT)

apturl==0.5.2
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
attrs==21.4.0
backcall==0.2.0
bcrypt==3.1.7
beautifulsoup4==4.11.1
bleach==5.0.0
blinker==1.4
Brlapi==0.7.0
certifi==2019.11.28
cffi==1.15.0
chardet==3.0.4
Click==7.0
colorama==0.4.3
command-not-found==0.3
cryptography==2.8
cupshelpers==1.0
dbus-python==1.2.16
debugpy==1.6.0
decorator==5.1.1
defer==1.0.6
defusedxml==0.7.1
distro==1.4.0
distro-info===0.23ubuntu1
duplicity==0.8.12.0
entrypoints==0.3
executing==0.8.3
fasteners==0.14.1
fastjsonschema==2.15.3
future==0.18.2
httplib2==0.14.0
idna==2.8
ipykernel==6.13.0
ipython==8.3.0
ipython-genutils==0.2.0
ipywidgets==7.7.0
jedi==0.18.1
Jinja2==3.1.2
jsonschema==4.4.0
jupyter==1.0.0
jupyter-client==7.3.0
jupyter-console==6.4.3
jupyter-core==4.10.0
jupyterlab-pygments==0.2.2
jupyterlab-widgets==1.1.0
keyring==18.0.1
language-selector==0.1
launchpadlib==1.10.13
lazr.restfulclient==0.14.2
lazr.uri==1.0.3
lockfile==0.12.2
louis==3.12.0
macaroonbakery==1.3.1
Mako==1.

In [8]:
import sys
print(sys.version)

3.9.12 (main, Apr 18 2022, 22:40:46) 
[GCC 9.4.0]


### GitHub Actions, Manual workflow и воспроизводимость кода

> Чтобы собственноручно убедиться в работоспособности предоставленного здесь метода, можете сделать [форк](https://github.com/web-standards-ru/dictionary/blob/main/git.md#fork) этого [репозитория](https://github.com/web-standards-ru/dictionary/blob/main/git.md#repository) или проделать необходимые шаги в собственном репозитории вручную.

Поставим перед собой задачу воспроизвести работоспособность кода используя GitHub Actions.

Оформим проверочный код в файл [test_game.py](test_game.py):

In [9]:

from game_v2 import random_predict
from result_game import random_binary_search, binary_search
from result_game import reveal_max_attempts, score_attempts

print()
print('random_predict (score_attempts):')
score_attempts(random_predict)
score_attempts(random_predict)
score_attempts(random_predict)
score_attempts(random_predict)
score_attempts(random_predict)

print()
print('random_binary_search (score_attempts):')
score_attempts(random_binary_search)
score_attempts(random_binary_search)
score_attempts(random_binary_search)
score_attempts(random_binary_search)
score_attempts(random_binary_search)

print()
print('binary_search (score_attempts):')
score_attempts(binary_search)
score_attempts(binary_search)
score_attempts(binary_search)
score_attempts(binary_search)
score_attempts(binary_search)

print()
print('random_predict (reveal_max_attempts):')
reveal_max_attempts(random_predict)
reveal_max_attempts(random_predict)
reveal_max_attempts(random_predict)
reveal_max_attempts(random_predict)
reveal_max_attempts(random_predict)

print()
print('random_binary_search (reveal_max_attempts):')
reveal_max_attempts(random_binary_search)
reveal_max_attempts(random_binary_search)
reveal_max_attempts(random_binary_search)
reveal_max_attempts(random_binary_search)
reveal_max_attempts(random_binary_search)

print()
print('binary_search (reveal_max_attempts):')
reveal_max_attempts(binary_search)
reveal_max_attempts(binary_search)
reveal_max_attempts(binary_search)
reveal_max_attempts(binary_search)
reveal_max_attempts(binary_search)


random_predict (score_attempts):
Search algorithm needs 103 attempts
Search algorithm needs 104 attempts
Search algorithm needs 102 attempts
Search algorithm needs 97 attempts
Search algorithm needs 98 attempts

random_binary_search (score_attempts):
Search algorithm needs 7 attempts
Search algorithm needs 7 attempts
Search algorithm needs 7 attempts
Search algorithm needs 7 attempts
Search algorithm needs 7 attempts

binary_search (score_attempts):
Search algorithm needs 5 attempts
Search algorithm needs 5 attempts
Search algorithm needs 5 attempts
Search algorithm needs 5 attempts
Search algorithm needs 5 attempts

random_predict (reveal_max_attempts):
Search algorithm needs maximum 695 attempts count
Search algorithm needs maximum 1230 attempts count
Search algorithm needs maximum 757 attempts count
Search algorithm needs maximum 622 attempts count
Search algorithm needs maximum 862 attempts count

random_binary_search (reveal_max_attempts):
Search algorithm needs maximum 15 attemp

7

В качестве отправной точки, воспользуемся (Automation) Manual workflow.

Создадим файл `.github/workflows/`[`reproduce.yml`](.github/workflows/reproduce.yml):
```
name: Воспроизводимость кода

on: workflow_dispatch

jobs:
  reproduce:
    name: Воспроизводимость кода
    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v1
    - name: Reproducing
      uses: ./reproduce
```

**А также:**

Файл `reproduce/`[`action.yml`](reproduce/action.yml):
```
runs:
  using: "docker"
  image: "Dockerfile"
```

Файл `reproduce/`[`Dockerfile`](reproduce/Dockerfile):
```
FROM python:3.9-slim

COPY requirements.txt /requirements.txt
RUN pip install --no-cache-dir -r /requirements.txt

COPY entrypoint.sh /entrypoint.sh
RUN chmod +x /entrypoint.sh
ENTRYPOINT ["/entrypoint.sh"]
```

Файл `reproduce/`[`entrypoint.sh`](reproduce/entrypoint.sh):
```
#!/bin/sh -l

python test_game.py

sh -c "echo"
sh -c "echo \#cat /etc/os-release"
sh -c "cat /etc/os-release"

sh -c "echo"
sh -c "echo \#uname -r"
sh -c "uname -r"

sh -c "echo"
sh -c "echo \#python --version"
sh -c "python --version"

echo
echo \#pip freeze --no-cache-dir
pip freeze --no-cache-dir
```

И файл `reproduce/`[`requirements.txt`](reproduce/requirements.txt):
```
numpy==1.22.3
```

В последний внесём необходимые нам строки из созданного ранее [requirements.txt](requirements.txt)

Перейдём к GitHub Actions, выберем workflow `Воспроизводимость кода` и запустим его через `Run workflow -> Run workflow`.

Дождёмся появления выполнения workflow `Воспроизводимость кода` и перейдём к нему.
По окончании выполнения (наличие зелёной галочки) выберем `Воспроизводимость кода` под `reproduce.yml`, раскроем шаг "Reproducing" и убедимся в том, что код выполнился как и ожидалось.